<a href="https://colab.research.google.com/github/yukiozzzz/gittest/blob/main/one_shot_talking_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title #**Setup**
!git lfs install
%cd /content
!git clone https://huggingface.co/camenduru/pocketsphinx-20.04-t4 pocketsphinx
%cd  /content/pocketsphinx
!sudo cmake --build build --target install
%cd /content
!git clone https://huggingface.co/camenduru/one-shot-talking-face-20.04-t4 one-shot-talking-face
%cd /content/one-shot-talking-face
!pip install -r /content/one-shot-talking-face/requirements.txt
!chmod 755 /content/one-shot-talking-face/OpenFace/FeatureExtraction
!mkdir /content/out
!apt install -qq libgtk2.0-0 jq -y
!pip install -q imageio-ffmpeg

Git LFS initialized.
/content
Cloning into 'pocketsphinx'...
remote: Enumerating objects: 1658, done.
remote: Total 1658 (delta 0), reused 0 (delta 0), pack-reused 1658 (from 1)
Receiving objects: 100% (1658/1658), 8.51 MiB | 7.97 MiB/s, done.
Resolving deltas: 100% (592/592), done.
/content/pocketsphinx
-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- SIZEOF_LONG="8" ; SIZEOF_LONG_LONG="8"
-- WORDS_BIGENDIAN="0"
-- SPHINX_DEBUG="" ; DEBUG_ENDIAN=""
-- FIXED_POINT="OFF" ; DEFAULT_RADIX="12"
-- Could NOT find Doxygen (missing: DOXYGEN_EXECUTABLE) 
-- Checking for module 'libpulse-simple'
--   Package 'libpulse-simple', required by 'virtual:world', not found
-- Checking for module 'portaudio-2.0'
--   Package 'portaudio-2.0', required by 'virtual:world', not found
-- MODELDIR="/usr/local/sha

In [2]:
#@title #**make movie**
import os, random, torchaudio
from IPython.display import HTML
from base64 import b64encode
from IPython.display import clear_output

def show_video(video_path, video_width = 256):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

#@markdown -Select wav_file_name from the one-shot-talking-face/samples/audios folder.\
#@markdown -Select image_file_name from the one-shot-talking-face/samples/imgs folder.

wav_file_name = 'obama2.wav' #@param {type:"string"}
image_file_name = 'paint.jpg' #@param {type:"string"}

wav_file = "/content/one-shot-talking-face/samples/audios/" + wav_file_name
image_file = "/content/one-shot-talking-face/samples/imgs/" + image_file_name

waveform, sample_rate = torchaudio.load(wav_file)
torchaudio.save(wav_file, waveform, sample_rate, encoding="PCM_S", bits_per_sample=16)

os.environ['wav_file'] = wav_file
os.environ['image_file'] = image_file
random_int = str(random.randint(1, 1000000))

!mkdir /content/train
!cp $wav_file /content/train/audio.wav
!cp $image_file /content/train/image.png

!pocketsphinx -phone_align yes single /content/train/audio.wav $text | jq '[.w[]|{word: (.t | ascii_upcase | sub("<S>"; "sil") | sub("<SIL>"; "sil") | sub("\\(2\\)"; "") | sub("\\(3\\)"; "") | sub("\\(4\\)"; "") | sub("\\[SPEECH\\]"; "SIL") | sub("\\[NOISE\\]"; "SIL")), phones: [.w[]|{ph: .t | sub("\\+SPN\\+"; "SIL") | sub("\\+NSN\\+"; "SIL"), bg: (.b*100)|floor, ed: (.b*100+.d*100)|floor}]}]' > /content/test.json
%cd /content/one-shot-talking-face
!python -B test_script.py --img_path /content/train/image.png --audio_path /content/train/audio.wav --phoneme_path /content/test.json --save_dir /content/train

os.environ['ran_num'] = random_int
!cp /content/train/image_audio.mp4 "/content/out/${ran_num}.mp4"

clear_output()

show_video(f"/content/out/{random_int}.mp4")

FileNotFoundError: [Errno 2] No such file or directory: '/content/out/757774.mp4'

In [ ]:
#@title #**Download movie** ( for google chrome)
from google.colab import files
file_path =f"/content/out/{random_int}.mp4"
files.download(file_path)